In [1]:
!pip install dotenv langchain  langchain-openai

In [2]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [3]:
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import time
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import accelerate
import transformers
import bitsandbytes as bnb

import os

import json

from datasets import load_dataset
import pandas as pd

import dotenv
from collections import Counter

from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
dotenv.load_dotenv('.env')

True

In [5]:
model_name = 'BioMistral/BioMistral-Safetensors'
results = {}

In [6]:
def clean_memory():
    for var in ['model', 'tokenizer', 'inputs', 'llama', 'inputs', 'output']:
        if var in globals():
            del globals()[var]

    if torch.cuda.is_available():
        torch.cuda.ipc_collect()
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()

    gc.collect()
    print('Memory is cleaned')

In [7]:
def print_memory():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1024**3
        reserved = torch.cuda.memory_reserved(0) / 1024**3
        print(f'VRAM allocated {allocated}gb, reserved {reserved}gb')
    else:
        print('No cuda')

In [8]:
clean_memory()
print_memory()

Memory is cleaned
VRAM allocated 0.0gb, reserved 0.0gb


In [9]:
clean_memory()
model_name = 'BioMistral/BioMistral-Safetensors'
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", attn_implementation="flash_attention_2") #Необходимо явно задать параметры, иначе будет другой размер и может не загрузится в видеокарту
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)
print_memory()

with torch.inference_mode():
    input_text = 'My head is sick. Which pill should I drink?'
    inputs = tokenizer(input_text, return_tensors='pt').to(model.device)
    
    start = time.time()
    output = model.generate(**inputs, repetition_penalty=1.2, max_new_tokens = 100)
    end = time.time()
    
    new_tokens = len(output[0]) - len(inputs.input_ids[0])
    tps = new_tokens / (end - start)
    print(f'BF16: new Tokens: {new_tokens}, tokens per second: {tps}')
    
    decoded_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(decoded_text)
    
    results['BioMistral-7B-BF16'] = {'tps': tps, 'memory': torch.cuda.memory_allocated(0)/1024**3}

Memory is cleaned


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


VRAM allocated 13.48877763748169gb, reserved 13.498046875gb
BF16: new Tokens: 49, tokens per second: 23.72660494451425
My head is sick. Which pill should I drink? (A) 10 mg of amoxicillin, (B) 250 mg of metronidazole, and (C) 400 mg of ciprofloxacin.


In [10]:
clean_memory()
print_memory()

Memory is cleaned
VRAM allocated 0.0079345703125gb, reserved 0.0390625gb


Парадоксально, но при bnb 4-bit пик памяти при загрузке может быть выше из-за временных буферов/конвертаций и особенностей размещения. Поэтому BF16 может “влезать”, а 4-bit — падать именно на этапе from_pretrained.
Заливаем модель в видеопамять через квантование на ОЗУ.

In [11]:
clean_memory()
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
)
model_name = 'BioMistral/BioMistral-Safetensors'

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map='auto', attn_implementation="flash_attention_2")
#model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)
print_memory()

model.eval()

with torch.inference_mode():

    input_text = 'My head is sick. Which pill should I drink?'
    inputs = tokenizer(input_text, return_tensors='pt').to(model.device)
    
    start = time.time()
    output = model.generate(**inputs, repetition_penalty=1.2, max_new_tokens = 100)
    end = time.time()
    
    new_tokens = len(output[0]) - len(inputs.input_ids[0])
    tps = new_tokens / (end - start)
    print(f'4BIT: new Tokens: {new_tokens}, tokens per second: {tps}')
    
    decoded_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(decoded_text)
    
    results['BioMistral-7B-4BIT'] = {'tps': tps, 'memory': torch.cuda.memory_allocated(0)/1024**3}

Memory is cleaned


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


VRAM allocated 4.153069019317627gb, reserved 4.240234375gb
4BIT: new Tokens: 57, tokens per second: 50.14993630447656
My head is sick. Which pill should I drink? (A) 10 mg of amitriptyline, (B) 25 mg of carbamazepine, (C) 300 mg of valproate, or (D) 400 mg of lamotrigine.


In [12]:
clean_memory()

model_name_gguf = 'BioMistral/BioMistral-7B-GGUF'

model_path = hf_hub_download(
    repo_id=model_name_gguf,
    filename='ggml-model-Q4_K_M.gguf',
    local_dir="./models"
)

print('File downloaded')

llama = Llama(
    model_path=model_path,
    n_gpu_layers=-1,
    n_ctx=32768,
    verbose=False
)

start = time.time()
output = llama.create_chat_completion(
    messages=[{'role': 'user', 'content': 'My head is sick. Which pill should I drink?'}],
    max_tokens=100,
    repeat_penalty=1.2,
)
end = time.time()
print(output)

new_tokens = output['usage']['completion_tokens']
tps = new_tokens / (end - start)
print(f'LLAMACPP: new Tokens: {new_tokens}, tokens per second: {tps}')

results['BioMistral-7B-LLAMACPP (GPU)'] = {'tps': tps, 'memory': 'uncountable'}

Memory is cleaned
File downloaded
{'id': 'chatcmpl-6bea617e-d44e-44bb-bd32-2083282bf8b0', 'object': 'chat.completion', 'created': 1771685449, 'model': 'models/ggml-model-Q4_K_M.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': ' I’m sorry to hear that. Can you please tell me more about how you are feeling? Are you experiencing a headache, fever or chills, nausea, vomiting, or any other symptoms?'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 19, 'completion_tokens': 44, 'total_tokens': 63}}
LLAMACPP: new Tokens: 44, tokens per second: 49.18111396199384


In [13]:
results

{'BioMistral-7B-BF16': {'tps': 23.72660494451425,
  'memory': 13.496713638305664},
 'BioMistral-7B-4BIT': {'tps': 50.14993630447656, 'memory': 4.15307092666626},
 'BioMistral-7B-LLAMACPP (GPU)': {'tps': 49.18111396199384,
  'memory': 'uncountable'}}

# Выводы по рантайму моделей

В текущем рантайме самый быстрый способ квантизованная в 4 бита модель? Она также дает наименьшее потребление памяти.

Но на других версиях bitandbytes, accelerate, transformers на квантизации в bitsandbytes получал OOM. Это связано с выделением большого количества видеопамяти в пике для квантизации в некоторых версиях библиотек.

Обходным путем была загрузка моделей в ОЗУ для квантизации, с ее дальнейшим переносом в видеопамять с помощью 

```python
model.to('cuda')
```

Но даже после переноса на gpu модель работала всего в 8 раза быстрее CPU (~4tps).

В итоге пришлось сбрать окружение в docker.

In [14]:
dataset = load_dataset('omi-health/medical-dialogue-to-soap-summary')

In [15]:
dataset['train'][45]['soap']

"S: The patient is a 79-year-old male with a history of hypertension, valvular heart disease, diabetes mellitus, and stage 4 chronic kidney disease, presenting with lower back pain. He reports previous treatments for bilateral large renal stones, including bilateral double J stent insertion, left extracorporeal shockwave lithotripsy, and FURS with laser stone fragmentation, which did not provide significant relief.\nO: Recent non-contrast CT scan revealed multiple bilateral renal stones with a total stone burden of 3.0 cm in the left kidney and 3.2 cm in the right kidney. The patient underwent simultaneous bilateral FURS and holmium laser lithotripsy under general anesthesia, which lasted 125 minutes. Surgical details include the use of a hydrophilic tip guidewire, a ureteral access sheath, and a Karl Storz flexible ureterorenoscope. Laser settings were 1.0–1.2 joules at 8-12 Hz, with 4200 and 4066 pulses for the left and right sides, respectively.\nA: The primary diagnosis is bilatera

In [16]:
examples = [d['soap'] for i, d in enumerate(dataset['train'].take(12))]

In [17]:
example = examples[1]
example

"S: The patient, a 21-month-old male, presented with weakness in his lower extremities and lumbar pain following a mild upper respiratory tract infection. Initial treatment with anti-inflammatory therapy for suspected transient hips arthritis was ineffective, leading to worsening pain and inability to walk.\nO: Hip ultrasound showed no joint effusion. Spine radiograph revealed slight reduction in the thickness of the L5 soma. MRI indicated increased T1 post-enhancement signals in the L4-L5 anulus, opposite end-plates of L4 and L5, adjacent soft tissues, and osteolytic area of the L5 pedicle. Blood tests showed elevated erythrocyte sedimentation rate (77 mm/h) and C-reactive protein (2.17 mg/dL), with normal white blood cell count, procalcitonin serum concentration, and Quantiferon TB-gold test. After 3 weeks of therapy, the patient developed leukopenia with severe neutropenia (white blood cell count at 5410/mm3).\nA: Primary diagnosis is Spondylodiscitis with associated osteomyelitis. 

In [18]:
def get_ner_prompt(text: str) -> str:
    return """
You are NER model. Find all named entities in text (between tags <text> and </text).

Allowed types ONLY:
- SYMPTOM: patient-reported symptoms or clinical manifestations (e.g., pain, weakness, fever, inability to walk).
- DIAGNOSIS: diseases/conditions (including suspected/differential diagnoses and infections).
- MEDICATION: specific drug names (e.g., meropenem, vancomycin). (Do NOT label tests or procedures.)
- DOSAGE: medication dose/regimen ONLY (e.g., "100 mg/kg/day", "in three doses", "IV", "once daily"). Must be tied to a MEDICATION in the same sentence/phrase.
- DURATION: time spans or timing of therapy/symptoms (e.g., "for 3 weeks", "after 3 weeks", "first week").
- SIDE_EFFECT: adverse effects/complications of treatment (e.g., leukopenia, neutropenia) when presented as developed during/after therapy.

STRICT EXCLUSIONS (do NOT extract):
- Imaging/procedures/tests: MRI, ultrasound, radiograph, CT, blood tests, Quantiferon, procalcitonin, etc.
- Lab values/measurements: ESR 77 mm/h, CRP 2.17 mg/dL, WBC 5410/mm3, normal ranges.
- Radiology/lab findings that are not symptoms (e.g., "increased T1 signals", "osteolytic area").
- Negated findings (e.g., "no joint effusion", "normal WBC") — do not extract them.

Output rules:
- Use exact text spans.
- Do not invent entities.
- Do not output duplicates.
- Split doses and medication titles (this is bad example of medication: meropenem (100 mg/kg/day in three doses))

Coverage checklist BEFORE finalizing the JSON:
1) Did you extract ALL SYMPTOM mentions (including worsening/functional inability)?
2) Did you extract ALL DIAGNOSIS mentions (including suspected/differential)?
3) For EACH antibiotic/drug: did you extract MEDICATION and its DOSAGE separately?
4) Did you extract ALL DURATION mentions related to therapy timing?
5) Did you avoid tests, imaging, lab values, and negated findings?
6) Didn't you mixed up dosage and duration?
7) Don't json have trailing comma after the last list element

<example>
<input>
 Doctor: Hello, what brings you in today?\nPatient: Hi, my 21-month-old son has been experiencing weakness in his lower extremities and lumbar pain after a mild upper respiratory tract infection.\nDoctor: I see. Did you consult any medical professional for this issue?\nPatient: Yes, we took him to the pediatric emergency department. They did a hip ultrasound, but they didn't find any joint effusion. They dismissed it as transient hips arthritis and treated him with anti-inflammatory therapy.\nDoctor: Did the therapy help with his condition?\nPatient: Unfortunately, no. His pain got worse, and now he's unable to walk. That's why we came back here.\nDoctor: We should definitely investigate further. We'll start by admitting him to the hospital. We'll need to perform a spine radiograph and an MRI to determine what's causing his condition. \nPatient: Okay, thank you.\n[After the tests]\nDoctor: I've reviewed the test results. The spine radiograph showed a slight reduction in the thickness of the L5 soma. The MRI revealed increased T1 post-enhancement signals in the L4-L5 anulus, the opposite end-plates of L4 and L5, the adjacent soft tissues, and the osteolytic area of the L5 pedicle. These findings suggest that your son has Spondylodiscitis with associated osteomyelitis.\nPatient: Oh no! What about his blood tests?\nDoctor: The blood tests showed an increase in the erythrocyte sedimentation rate (77 mm/h) and C-reactive protein (2.17 mg/dL). However, his white blood cell count, procalcitonin serum concentration, and Quantiferon TB-gold test are all within normal ranges. \nPatient: What's the treatment plan for his condition?\nDoctor: We'll start him on broad-spectrum intravenous therapy, which includes meropenem (100 mg/kg/day in three doses) and vancomycin (40 mg/kg/day in three doses). We'll continue the anti-inflammatory treatment for the first week and then stop it once his symptoms have resolved and he can walk normally again.\nPatient: How long will the treatment last?\nDoctor: The treatment may last for a few weeks. We'll closely monitor his progress and adjust the treatment as needed.\nPatient: Alright, thank you, doctor.\n[After 3 weeks of therapy]\nDoctor: I wanted to update you on your son's condition. Unfortunately, he has developed leukopenia with severe neutropenia. His white blood cell count dropped to its lowest at 5410/mm3.\nPatient: Oh no, is there anything we can do?\nDoctor: We'll continue to closely monitor his condition and tailor his treatment as necessary. In the meantime, it's important for you to keep an eye on any changes in his symptoms and report them to us immediately. We'll do our best to help him recover.\nPatient: Thank you, doctor. We appreciate your help. 
Use meropenem (100 mg/kg/day in three doses)
</input>
<output>
```json
[
  {{"entity":"weakness in his lower extremities","type":"SYMPTOM"}},
  {{"entity":"lumbar pain","type":"SYMPTOM"}},
  {{"entity":"upper respiratory tract infection","type":"DIAGNOSIS"}},
  {{"entity":"anti-inflammatory therapy","type":"MEDICATION"}},
  {{"entity":"meropenem","type":"MEDICATION"}},
  {{"entity":"100 mg/kg/day in three doses","type":"DOSAGE"}}
]
```
</output>
</example>

Think step by step.

Text:
<input>
{}
</input>

Your answer in json:
""".format(text)

In [19]:
df = pd.DataFrame({'soap': examples})
df['ner_prompt'] = df['soap'].apply(get_ner_prompt)

df

,soap,ner_prompt
0,S: The patient's mother reports that her 13-ye...,\nYou are NER model. Find all named entities i...
1,"S: The patient, a 21-month-old male, presented...",\nYou are NER model. Find all named entities i...
2,"S: Patient reports experiencing fatigue, night...",\nYou are NER model. Find all named entities i...
3,"S: Patient D, a 60-year-old African American m...",\nYou are NER model. Find all named entities i...
4,"S: The patient, a married woman with a 7-year ...",\nYou are NER model. Find all named entities i...
5,"S: The patient, a long-term sufferer of acrome...",\nYou are NER model. Find all named entities i...
6,S: The patient reports a history of abdominal ...,\nYou are NER model. Find all named entities i...
7,"S: The patient, who has been experiencing righ...",\nYou are NER model. Find all named entities i...
8,"S: The patient, with a past history of sigmoid...",\nYou are NER model. Find all named entities i...
9,"S: The patient, with a history of asthma, was ...",\nYou are NER model. Find all named entities i...


# Выбор модели для NER задачи

В качестве NER модель biomistral не подошла, так как она не instruct. Выбрал Mistral-7B-Instruct. Для оптимизиации использую flash attention 2.
Благодаря flash attention:
- Снизил потребление памяти при больших контекстах
- Минимальное ускорение на инференсе

Для дальнейших экспериментов с бэтчингом сразу же настраиваем падовый токен.

In [20]:
def get_ner_model():
    model_name = 'mistralai/Mistral-7B-Instruct-v0.3'

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.float16,
    )
    
    model = AutoModelForCausalLM.from_pretrained(
                                            model_name,
                                            torch_dtype=torch.float16,
                                            device_map='auto',
                                            quantization_config=bnb_config,
                                            attn_implementation="flash_attention_2"
                                            ) #Необходимо явно задать параметры, иначе будет другой размер и может не загрузится в видеокарту
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id
    tokenizer.padding_side = "left"
    
    model.eval()

    

    return model, tokenizer

In [21]:
def get_ner_answer(model_output: str):
    try:
        index = model_output.rfind("```json")
        if index == -1:
            print(model_output)
            raise Exception("Cannot find ```json tag")
        model_output = model_output[index+7:-4]
        return json.loads(model_output)
    except Exception as e:
        print(model_output)
        print(f'cannot parse json output: {e}')
        return []

In [22]:
#uncomment it on the first run
clean_memory()
ner_model, ner_tokenizer = get_ner_model()

Memory is cleaned


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
clean_memory()

input_text = get_ner_prompt(example)

print_memory()

with torch.inference_mode():

    inputs = ner_tokenizer(input_text, return_tensors='pt').to(ner_model.device)
    
    start = time.time()
    output = ner_model.generate(**inputs, max_new_tokens = 4096, temperature=0.0,)
    end = time.time()
    
    new_tokens = len(output[0]) - len(inputs.input_ids[0])
    tps = new_tokens / (end - start)
    print(f'BF16: new Tokens: {new_tokens}, tokens per second: {tps}')
    
    decoded_text = ner_tokenizer.decode(output[0], skip_special_tokens=True)
    print(decoded_text)
    print(get_ner_answer(decoded_text))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Memory is cleaned
VRAM allocated 4.164787769317627gb, reserved 4.240234375gb
BF16: new Tokens: 422, tokens per second: 46.401207545460366

You are NER model. Find all named entities in text (between tags <text> and </text).

Allowed types ONLY:
- SYMPTOM: patient-reported symptoms or clinical manifestations (e.g., pain, weakness, fever, inability to walk).
- DIAGNOSIS: diseases/conditions (including suspected/differential diagnoses and infections).
- MEDICATION: specific drug names (e.g., meropenem, vancomycin). (Do NOT label tests or procedures.)
- DOSAGE: medication dose/regimen ONLY (e.g., "100 mg/kg/day", "in three doses", "IV", "once daily"). Must be tied to a MEDICATION in the same sentence/phrase.
- DURATION: time spans or timing of therapy/symptoms (e.g., "for 3 weeks", "after 3 weeks", "first week").
- SIDE_EFFECT: adverse effects/complications of treatment (e.g., leukopenia, neutropenia) when presented as developed during/after therapy.

STRICT EXCLUSIONS (do NOT extract):
- 

In [24]:
clean_memory()

Memory is cleaned


In [25]:
print_memory()

VRAM allocated 4.164787769317627gb, reserved 4.240234375gb


In [26]:
def batch_ner(prompts, model, tokenizer):
    inputs = tokenizer(
        prompts,
        return_tensors='pt',
        padding=True,
        max_length=10240,
        truncation=True
    ).to('cuda')

    out = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
    )

    texts = tokenizer.batch_decode(out, skip_special_tokens=True)
    return texts
    

In [27]:
#SMART BATCHING
def batch_process_dataframe(df, batch_size_max, size_diff_to_batch, model, tokenizer):
    i = 0
    while i < len(df):
        j = i
        idx = []

        batch_size = 0

        while j < len(df):
            if df['ner_prompt_len'][df.index[i]] / float(df['ner_prompt_len'][df.index[j]]) >= size_diff_to_batch:
                batch_size = batch_size + 1
                idx.append(df.index[j])
                j = j + 1
                if batch_size >= batch_size_max:
                    break
            else:
                break

        batch = df.loc[idx]
        sizes = batch['ner_prompt_len'].tolist()
        print(sizes)

        prompts = batch['ner_prompt'].tolist()
        ner_by_local_model = batch_ner(prompts, model, tokenizer)
        ner_by_local_model_obj = [get_ner_answer(i) for i in ner_by_local_model]

        df.loc[idx, "ner_result"] = ner_by_local_model
        df.loc[idx, "ner_parsed"] = pd.Series(ner_by_local_model_obj, index=idx, dtype="object")#ner_by_local_model_obj

        i = j

In [28]:
def get_total_tokens_in_prompt(tokenizer):
    def get_total_tokens_in_prompt(prompt):
        inputs = tokenizer(prompt)
        return len(inputs['input_ids'])
    return get_total_tokens_in_prompt

In [29]:
df['ner_prompt_len'] = df['ner_prompt'].apply(get_total_tokens_in_prompt(ner_tokenizer))
df = df.sort_values(by='ner_prompt_len')
df

,soap,ner_prompt,ner_prompt_len
3,"S: Patient D, a 60-year-old African American m...",\nYou are NER model. Find all named entities i...,1744
10,"S: The patient, a postpartum woman with a hist...",\nYou are NER model. Find all named entities i...,1919
0,S: The patient's mother reports that her 13-ye...,\nYou are NER model. Find all named entities i...,1924
1,"S: The patient, a 21-month-old male, presented...",\nYou are NER model. Find all named entities i...,1965
6,S: The patient reports a history of abdominal ...,\nYou are NER model. Find all named entities i...,1973
4,"S: The patient, a married woman with a 7-year ...",\nYou are NER model. Find all named entities i...,1973
7,"S: The patient, who has been experiencing righ...",\nYou are NER model. Find all named entities i...,1989
11,"S: The patient, diagnosed with infantile-onset...",\nYou are NER model. Find all named entities i...,2002
9,"S: The patient, with a history of asthma, was ...",\nYou are NER model. Find all named entities i...,2013
2,"S: Patient reports experiencing fatigue, night...",\nYou are NER model. Find all named entities i...,2020


In [30]:
time_batch_logs = []

In [31]:
start = time.time()
batch_size=1
size_diff_to_batch=0.99
batch_process_dataframe(df, batch_size_max=batch_size, size_diff_to_batch=size_diff_to_batch, model=ner_model, tokenizer=ner_tokenizer)
end = time.time()
log = f'Time with batch {batch_size} and max diff {size_diff_to_batch} for df len {len(df)}: {end-start} seconds.'
print(log)
time_batch_logs.append(log)

start = time.time()
batch_size=2
size_diff_to_batch=0.99
batch_process_dataframe(df, batch_size_max=batch_size, size_diff_to_batch=size_diff_to_batch, model=ner_model, tokenizer=ner_tokenizer)
end = time.time()
log = f'Time with batch {batch_size} and max diff {size_diff_to_batch} for df len {len(df)}: {end-start} seconds.'
print(log)
time_batch_logs.append(log)

start = time.time()
batch_size=5
size_diff_to_batch=0.99
batch_process_dataframe(df, batch_size_max=batch_size, size_diff_to_batch=size_diff_to_batch, model=ner_model, tokenizer=ner_tokenizer)
end = time.time()
log = f'Time with batch {batch_size} and max diff {size_diff_to_batch} for df len {len(df)}: {end-start} seconds.'
print(log)
time_batch_logs.append(log)

start = time.time()
batch_size=7
size_diff_to_batch=0.99
batch_process_dataframe(df, batch_size_max=batch_size, size_diff_to_batch=size_diff_to_batch, model=ner_model, tokenizer=ner_tokenizer)
end = time.time()
log = f'Time with batch {batch_size} and max diff {size_diff_to_batch} for df len {len(df)}: {end-start} seconds.'
print(log)
time_batch_logs.append(log)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[1744]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[1919]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

[1989, 2002]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[2013, 2020, 2024]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[2035]
Time with batch 7 and max diff 0.99 for df len 12: 190.20998764038086 seconds.


In [32]:
for log in time_batch_logs:
    print(log)

Time with batch 1 and max diff 0.99 for df len 12: 102.66518807411194 seconds.
Time with batch 2 and max diff 0.99 for df len 12: 222.52455234527588 seconds.
Time with batch 5 and max diff 0.99 for df len 12: 186.31592178344727 seconds.
Time with batch 7 and max diff 0.99 for df len 12: 190.20998764038086 seconds.


# Batching

Работа с бэтчингом в наивном режиме совсем не работала. Для усоркния бэтчинга попытался сортировать промпты по длине в токенах.
Этого оказалось мало. Стал отбирать размер батча динамическии.
Но даже в таком формате на этой задаче оказалось дешевле 1 обсчитывать промты.

Это связано с тем, что выход по промптам тоже отилчается по размеру и приходится пересчитывать тензоры всех элементов батча по размеру самого длинного выхода.

In [33]:
df

,soap,ner_prompt,ner_prompt_len,ner_result,ner_parsed
3,"S: Patient D, a 60-year-old African American m...",\nYou are NER model. Find all named entities i...,1744,\nYou are NER model. Find all named entities i...,"[{'entity': 'Patient D', 'type': 'PATIENT'}, {..."
10,"S: The patient, a postpartum woman with a hist...",\nYou are NER model. Find all named entities i...,1919,\nYou are NER model. Find all named entities i...,"[{'entity': 'postpartum woman', 'type': 'PATIE..."
0,S: The patient's mother reports that her 13-ye...,\nYou are NER model. Find all named entities i...,1924,\nYou are NER model. Find all named entities i...,"[{'entity': 'attention deficit disorder', 'typ..."
1,"S: The patient, a 21-month-old male, presented...",\nYou are NER model. Find all named entities i...,1965,\nYou are NER model. Find all named entities i...,"[{'entity': '21-month-old male', 'type': 'PATI..."
6,S: The patient reports a history of abdominal ...,\nYou are NER model. Find all named entities i...,1973,\nYou are NER model. Find all named entities i...,"[{'entity': 'abdominal pain', 'type': 'SYMPTOM..."
4,"S: The patient, a married woman with a 7-year ...",\nYou are NER model. Find all named entities i...,1973,\nYou are NER model. Find all named entities i...,"[{'entity': '7-year history of infertility', '..."
7,"S: The patient, who has been experiencing righ...",\nYou are NER model. Find all named entities i...,1989,\nYou are NER model. Find all named entities i...,"[{'entity': 'right ankle swelling', 'type': 'S..."
11,"S: The patient, diagnosed with infantile-onset...",\nYou are NER model. Find all named entities i...,2002,\nYou are NER model. Find all named entities i...,[{'entity': 'infantile-onset hypophosphatasia'...
9,"S: The patient, with a history of asthma, was ...",\nYou are NER model. Find all named entities i...,2013,\nYou are NER model. Find all named entities i...,"[{'entity': 'asthma', 'type': 'DIAGNOSIS'}, {'..."
2,"S: Patient reports experiencing fatigue, night...",\nYou are NER model. Find all named entities i...,2020,\nYou are NER model. Find all named entities i...,"[{'entity': 'fatigue', 'type': 'SYMPTOM'}, {'e..."


In [34]:
print(df['ner_parsed'][3])

[{'entity': 'Patient D', 'type': 'PATIENT'}, {'entity': '60-year-old African American male', 'type': 'PATIENT'}, {'entity': 'prostate cancer', 'type': 'DIAGNOSIS'}, {'entity': 'family history significant for prostate cancer', 'type': 'DIAGNOSIS'}, {'entity': '62-year-old brother', 'type': 'PATIENT'}, {'entity': 'radiation', 'type': 'TREATMENT'}, {'entity': 'increased risk for prostate cancer', 'type': 'RISK'}, {'entity': 'ethnicity and family history (first-degree relative diagnosed before age 65)', 'type': 'RISK'}, {'entity': 'PSA testing', 'type': 'TEST'}, {'entity': 'varying recommendations from different health organizations', 'type': 'DIAGNOSIS'}, {'entity': 'detailed conversation about PSA testing', 'type': 'PLAN'}, {'entity': 'research further and prepare any questions for a follow-up discussion', 'type': 'PLAN'}, {'entity': 'report any new symptoms or concerns in the interim', 'type': 'PLAN'}]


In [35]:
llm = ChatOpenAI(
    api_key=os.environ['API_KEY'],
    base_url=os.environ['API_BASE_URL'],
    temperature=0.0,
    model='qwen-3-32b'
)
golden_parse_iter = 0

In [36]:
def get_golden_ner_value(query:str):
    try:
        global golden_parse_iter
        golden_parse_iter = golden_parse_iter + 1
        start = time.time()
        llm_resp = (ChatPromptTemplate.from_messages([HumanMessage(query)]) | llm | StrOutputParser()).invoke({})
        obj = get_ner_answer(llm_resp)
        end = time.time()
        print(f'Calculating golden result iteration {golden_parse_iter} for time {end - start}')
        return obj
    except e as Exception:
        print(f'error from llm: {e}')
        return []

In [37]:
get_golden_ner_value(df['ner_prompt'][3])

Calculating golden result iteration 1 for time 0.9627058506011963


[{'entity': 'prostate cancer', 'type': 'DIAGNOSIS'},
 {'entity': 'PSA testing', 'type': 'MEDICATION'}]

In [38]:
golden_parse_iter = 0
df["ner_golden_parsed"] = df['ner_prompt'].apply(get_golden_ner_value)


Calculating golden result iteration 1 for time 0.47370290756225586
Calculating golden result iteration 2 for time 1.5381362438201904
Calculating golden result iteration 3 for time 2.9520373344421387
Calculating golden result iteration 4 for time 2.7644662857055664
Calculating golden result iteration 5 for time 1.1795222759246826
Calculating golden result iteration 6 for time 1.686462640762329
Calculating golden result iteration 7 for time 3.484342575073242
Calculating golden result iteration 8 for time 2.6579749584198
Calculating golden result iteration 9 for time 3.3128933906555176
Calculating golden result iteration 10 for time 2.9761483669281006
Calculating golden result iteration 11 for time 2.3074140548706055
Calculating golden result iteration 12 for time 2.704714775085449


In [39]:
df

,soap,ner_prompt,ner_prompt_len,ner_result,ner_parsed,ner_golden_parsed
3,"S: Patient D, a 60-year-old African American m...",\nYou are NER model. Find all named entities i...,1744,\nYou are NER model. Find all named entities i...,"[{'entity': 'Patient D', 'type': 'PATIENT'}, {...","[{'entity': 'prostate cancer', 'type': 'DIAGNO..."
10,"S: The patient, a postpartum woman with a hist...",\nYou are NER model. Find all named entities i...,1919,\nYou are NER model. Find all named entities i...,"[{'entity': 'postpartum woman', 'type': 'PATIE...","[{'entity': 'postpartum woman', 'type': 'DIAGN..."
0,S: The patient's mother reports that her 13-ye...,\nYou are NER model. Find all named entities i...,1924,\nYou are NER model. Find all named entities i...,"[{'entity': 'attention deficit disorder', 'typ...","[{'entity': 'speech and developmental delays',..."
1,"S: The patient, a 21-month-old male, presented...",\nYou are NER model. Find all named entities i...,1965,\nYou are NER model. Find all named entities i...,"[{'entity': '21-month-old male', 'type': 'PATI...",[{'entity': 'weakness in his lower extremities...
6,S: The patient reports a history of abdominal ...,\nYou are NER model. Find all named entities i...,1973,\nYou are NER model. Find all named entities i...,"[{'entity': 'abdominal pain', 'type': 'SYMPTOM...","[{'entity': 'abdominal pain', 'type': 'SYMPTOM..."
4,"S: The patient, a married woman with a 7-year ...",\nYou are NER model. Find all named entities i...,1973,\nYou are NER model. Find all named entities i...,"[{'entity': '7-year history of infertility', '...","[{'entity': 'irregular menstruation', 'type': ..."
7,"S: The patient, who has been experiencing righ...",\nYou are NER model. Find all named entities i...,1989,\nYou are NER model. Find all named entities i...,"[{'entity': 'right ankle swelling', 'type': 'S...","[{'entity': 'right ankle swelling', 'type': 'S..."
11,"S: The patient, diagnosed with infantile-onset...",\nYou are NER model. Find all named entities i...,2002,\nYou are NER model. Find all named entities i...,[{'entity': 'infantile-onset hypophosphatasia'...,[{'entity': 'infantile-onset hypophosphatasia'...
9,"S: The patient, with a history of asthma, was ...",\nYou are NER model. Find all named entities i...,2013,\nYou are NER model. Find all named entities i...,"[{'entity': 'asthma', 'type': 'DIAGNOSIS'}, {'...","[{'entity': 'unresponsive', 'type': 'SYMPTOM'}..."
2,"S: Patient reports experiencing fatigue, night...",\nYou are NER model. Find all named entities i...,2020,\nYou are NER model. Find all named entities i...,"[{'entity': 'fatigue', 'type': 'SYMPTOM'}, {'e...","[{'entity': 'fatigue', 'type': 'SYMPTOM'}, {'e..."


In [40]:
df['ner_golden_parsed'][3][1]['type']

'MEDICATION'

In [41]:
def get_prepared_ner_array(obj):
    res = []
    for elem in obj:
        entity = elem['entity']
        ent_type = elem['type']
        if ent_type not in ('SYMPTOM', 'DIAGNOSIS', 'MEDICATION', 'DOSAGE', 'DURATION', 'SIDE_EFFECT'):
            continue
        splited_entity = entity.split(' ')
        for se in splited_entity:
            res.append((se.lower(), ent_type))
    return res

def get_tp_fp_fn(row):
    ner_parsed = get_prepared_ner_array(row['ner_parsed'])
    ner_golden_parsed = get_prepared_ner_array(row['ner_golden_parsed'])

    gold_counter = Counter(ner_golden_parsed)
    pred_counter = Counter(ner_parsed)

    all_keys = set(gold_counter.keys()) | set(pred_counter.keys())

    tp = 0.0
    fp = 0.0
    fn = 0.0

    for key in all_keys:
        gold_count = gold_counter.get(key, 0.0)
        pred_count = pred_counter.get(key, 0.0)

        tp += min(gold_count, pred_count)

        if pred_count > gold_count:
            fp += pred_count - gold_count
        if gold_count > pred_count:
            fn += gold_count - pred_count

    return (tp, fp, fn)

def calc_precision(row):
    (tp, fp, fn) = get_tp_fp_fn(row)
    return (tp / (tp + fp)) if (tp + fp) > 0 else 0.0

def calc_recall(row):
    (tp, fp, fn) = get_tp_fp_fn(row)
    return (tp / (tp + fn)) if (tp + fn) > 0 else 0.0
    

In [42]:
df['precision'] = df.apply(calc_precision, axis=1)
df['recall'] = df.apply(calc_recall, axis=1)

In [43]:
df

,soap,ner_prompt,ner_prompt_len,ner_result,ner_parsed,ner_golden_parsed,precision,recall
3,"S: Patient D, a 60-year-old African American m...",\nYou are NER model. Find all named entities i...,1744,\nYou are NER model. Find all named entities i...,"[{'entity': 'Patient D', 'type': 'PATIENT'}, {...","[{'entity': 'prostate cancer', 'type': 'DIAGNO...",0.142857,0.500000
10,"S: The patient, a postpartum woman with a hist...",\nYou are NER model. Find all named entities i...,1919,\nYou are NER model. Find all named entities i...,"[{'entity': 'postpartum woman', 'type': 'PATIE...","[{'entity': 'postpartum woman', 'type': 'DIAGN...",0.268293,0.611111
0,S: The patient's mother reports that her 13-ye...,\nYou are NER model. Find all named entities i...,1924,\nYou are NER model. Find all named entities i...,"[{'entity': 'attention deficit disorder', 'typ...","[{'entity': 'speech and developmental delays',...",0.782609,0.750000
1,"S: The patient, a 21-month-old male, presented...",\nYou are NER model. Find all named entities i...,1965,\nYou are NER model. Find all named entities i...,"[{'entity': '21-month-old male', 'type': 'PATI...",[{'entity': 'weakness in his lower extremities...,0.702128,0.733333
6,S: The patient reports a history of abdominal ...,\nYou are NER model. Find all named entities i...,1973,\nYou are NER model. Find all named entities i...,"[{'entity': 'abdominal pain', 'type': 'SYMPTOM...","[{'entity': 'abdominal pain', 'type': 'SYMPTOM...",0.112360,0.714286
4,"S: The patient, a married woman with a 7-year ...",\nYou are NER model. Find all named entities i...,1973,\nYou are NER model. Find all named entities i...,"[{'entity': '7-year history of infertility', '...","[{'entity': 'irregular menstruation', 'type': ...",0.428571,0.450000
7,"S: The patient, who has been experiencing righ...",\nYou are NER model. Find all named entities i...,1989,\nYou are NER model. Find all named entities i...,"[{'entity': 'right ankle swelling', 'type': 'S...","[{'entity': 'right ankle swelling', 'type': 'S...",0.878788,0.743590
11,"S: The patient, diagnosed with infantile-onset...",\nYou are NER model. Find all named entities i...,2002,\nYou are NER model. Find all named entities i...,[{'entity': 'infantile-onset hypophosphatasia'...,[{'entity': 'infantile-onset hypophosphatasia'...,0.338710,0.617647
9,"S: The patient, with a history of asthma, was ...",\nYou are NER model. Find all named entities i...,2013,\nYou are NER model. Find all named entities i...,"[{'entity': 'asthma', 'type': 'DIAGNOSIS'}, {'...","[{'entity': 'unresponsive', 'type': 'SYMPTOM'}...",0.298246,0.435897
2,"S: Patient reports experiencing fatigue, night...",\nYou are NER model. Find all named entities i...,2020,\nYou are NER model. Find all named entities i...,"[{'entity': 'fatigue', 'type': 'SYMPTOM'}, {'e...","[{'entity': 'fatigue', 'type': 'SYMPTOM'}, {'e...",0.589744,0.575000


In [44]:
mean_precision = df['precision'].mean()
mean_recall = df['recall'].mean()
print(f'Mean precision {mean_precision} ::: Mean Recall {mean_recall}')

Mean precision 0.5070967552489078 ::: Mean Recall 0.586662942177648


## Эксперимент на большом датасете с лучшими параметрами

In [45]:
examples = [d['soap'] for i, d in enumerate(dataset['train'].take(1000))]
df = pd.DataFrame({'soap': examples})
df['ner_prompt'] = df['soap'].apply(get_ner_prompt)
df['ner_prompt_len'] = df['ner_prompt'].apply(get_total_tokens_in_prompt(ner_tokenizer))
df = df.sort_values(by='ner_prompt_len')
df

,soap,ner_prompt,ner_prompt_len
481,"S: MH, a volunteer in a cognitive study, repor...",\nYou are NER model. Find all named entities i...,1719
3,"S: Patient D, a 60-year-old African American m...",\nYou are NER model. Find all named entities i...,1744
18,S: The 6-year-old patient reports difficulty c...,\nYou are NER model. Find all named entities i...,1768
646,S: The patient reports a recent unexplained we...,\nYou are NER model. Find all named entities i...,1774
461,S: The patient presents with a chief complaint...,\nYou are NER model. Find all named entities i...,1780
...,...,...,...
144,S: The patient is a 19-year-old male presentin...,\nYou are NER model. Find all named entities i...,2203
358,"S: The patient, a long-term sufferer of seropo...",\nYou are NER model. Find all named entities i...,2209
334,"S: The patient, previously diagnosed with IgA ...",\nYou are NER model. Find all named entities i...,2212
661,S: The patient is a 20-year-old Brazilian male...,\nYou are NER model. Find all named entities i...,2217


In [46]:
batch_size=1
size_diff_to_batch=0.99
batch_process_dataframe(df, batch_size_max=batch_size, size_diff_to_batch=size_diff_to_batch, model=ner_model, tokenizer=ner_tokenizer)
golden_parse_iter = 0
df["ner_golden_parsed"] = df['ner_prompt'].apply(get_golden_ner_value)
df

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[1719]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[1744]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[2217]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[
  {"entity":"primary hyperparathyroidism (PHPT)","type":"DIAGNOSIS"},
  {"entity":"partial parathyroidectomy","type":"PROCEDURE"},
  {"entity":"asymptomatic","type":"SYMPTOM"},
  {"entity":"high serum PTH levels","type":"DIAGNOSIS"},
  {"entity":"progressive renal impairment","type":"DIAGNOSIS"},
  {"entity":"urinary symptoms","type":"SYMPTOM"},
  {"entity":"bilateral nephrocalcinosis (NC)","type":"DIAGNOSIS"},
  {"entity":"nephrolithiasis","type":"DIAGNOSIS"},
  {"entity":"serum parathormone (PTH)","type":"TEST"},
  {"entity":"227 pg/mL","type":"LAB_VALUE"},
  {"entity":"374 pg/mL","type":"LAB_VALUE"},
  {"entity":"serum creatinine","type":"TEST"},
  {"entity":"1.67 mg/dL","type":"LAB_VALUE"},
  {"entity":"2.25 mg/dL","type":"LAB_VALUE"},
  {"entity":"Glomerular filtration rate (GFR)","type":"TEST"},
  {"entity":"40 mL/min/1.73m2","type":"LAB_VALUE"},
  {"entity":"serum calcium and phosphorus","type":"TEST"},
  {"entity":"Normal","type":"LAB_VALUE"},
  {"entity":"Hypercalciuria","t

,soap,ner_prompt,ner_prompt_len,ner_result,ner_parsed,ner_golden_parsed
481,"S: MH, a volunteer in a cognitive study, repor...",\nYou are NER model. Find all named entities i...,1719,\nYou are NER model. Find all named entities i...,"[{'entity': 'MH', 'type': 'PATIENT'}, {'entity...",[]
3,"S: Patient D, a 60-year-old African American m...",\nYou are NER model. Find all named entities i...,1744,\nYou are NER model. Find all named entities i...,"[{'entity': 'Patient D', 'type': 'PATIENT'}, {...","[{'entity': 'prostate cancer', 'type': 'DIAGNO..."
18,S: The 6-year-old patient reports difficulty c...,\nYou are NER model. Find all named entities i...,1768,\nYou are NER model. Find all named entities i...,"[{'entity': 'difficulty chewing food', 'type':...","[{'entity': 'difficulty chewing food', 'type':..."
646,S: The patient reports a recent unexplained we...,\nYou are NER model. Find all named entities i...,1774,\nYou are NER model. Find all named entities i...,[{'entity': 'unexplained weight loss of 10 kg'...,[{'entity': 'unexplained weight loss of 10 kg'...
461,S: The patient presents with a chief complaint...,\nYou are NER model. Find all named entities i...,1780,\nYou are NER model. Find all named entities i...,"[{'entity': 'cough', 'type': 'SYMPTOM'}, {'ent...","[{'entity': 'cough', 'type': 'SYMPTOM'}, {'ent..."
...,...,...,...,...,...,...
144,S: The patient is a 19-year-old male presentin...,\nYou are NER model. Find all named entities i...,2203,\nYou are NER model. Find all named entities i...,"[{'entity': 'polydipsia', 'type': 'SYMPTOM'}, ...","[{'entity': 'polydipsia', 'type': 'SYMPTOM'}, ..."
358,"S: The patient, a long-term sufferer of seropo...",\nYou are NER model. Find all named entities i...,2209,\nYou are NER model. Find all named entities i...,[{'entity': 'seropositive rheumatoid arthritis...,[{'entity': 'involuntary shaking head movement...
334,"S: The patient, previously diagnosed with IgA ...",\nYou are NER model. Find all named entities i...,2212,\nYou are NER model. Find all named entities i...,[{'entity': 'IgA κ-type smoldering multiple my...,[{'entity': 'IgA κ-type smoldering multiple my...
661,S: The patient is a 20-year-old Brazilian male...,\nYou are NER model. Find all named entities i...,2217,\nYou are NER model. Find all named entities i...,[],[{'entity': 'increased serum creatinine level'...


In [47]:
df['precision'] = df.apply(calc_precision, axis=1)
df['recall'] = df.apply(calc_recall, axis=1)
mean_precision = df['precision'].mean()
mean_recall = df['recall'].mean()
print(f'Mean precision {mean_precision} ::: Mean Recall {mean_recall}')

Mean precision 0.47844333756785123 ::: Mean Recall 0.5907857339150462


# Выводы

NER на промтах на 7B показал точность всего в 48% и recall 59%.
Заметил, что он часто выдает незапрошенные названия в JSON и на этом тоже теряется recall.

Видимо, более правильные варианты:
- Fine Tune biomistral на инструкции через lora.
- Использование BERT с дообучением на разметку.

Ниже пытался заставить фиксить JSON, но текущая модель ошибалась, в отличии от qwen3-coder.

In [107]:
def get_ner_fixing_prompt(output_to_fix, bad_json_keys):
    bad_keys = ', '.join(bad_json_keys)
    return '''{}
You returned correct json, but it contains not requested types: {}
It must contain only types: 'SYMPTOM', 'DIAGNOSIS', 'MEDICATION', 'DOSAGE', 'DURATION', 'SIDE_EFFECT'.
Fix json, saving only correct types. In case of mistake you can:
- Delete json array element in case it has incorrect symantic.
- Rename type, if you make mistake.

Answer only with json and stop. Do not add any additional Information.

Your answer in json:
'''.format(output_to_fix, bad_keys)

In [110]:
def ner_with_json_fixes(orig_prompt):
    with torch.inference_mode():
        inputs = ner_tokenizer(orig_prompt, return_tensors='pt').to(ner_model.device)
        output = ner_model.generate(**inputs, max_new_tokens = 10000, do_sample=False)
        output_text = ner_tokenizer.decode(output[0], skip_special_tokens=True)
        obj = get_ner_answer(output_text)

        bad_json_keys = []
        for item in obj:
            if item['type'].upper() not in ('SYMPTOM', 'DIAGNOSIS', 'MEDICATION', 'DOSAGE', 'DURATION', 'SIDE_EFFECT'):
                bad_json_keys.append(item['type'])

        if len(bad_json_keys) == 0:
            return obj

        fixing_prompt = get_ner_fixing_prompt(output_text, bad_json_keys)
        inputs = ner_tokenizer(fixing_prompt, return_tensors='pt').to(ner_model.device)
        output = ner_model.generate(**inputs, max_new_tokens = 300, do_sample=False)
        output_text = ner_tokenizer.decode(output[0], skip_special_tokens=True)
        obj = get_ner_answer(output_text)

        print(fixing_prompt)
        return obj

In [111]:
df[0:2]['ner_prompt'].apply(ner_with_json_fixes)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



cannot parse json output: Expecting value: line 1 column 1 (char 0)

You are NER model. Find all named entities in text (between tags <text> and </text).

Allowed types ONLY:
- SYMPTOM: patient-reported symptoms or clinical manifestations (e.g., pain, weakness, fever, inability to walk).
- DIAGNOSIS: diseases/conditions (including suspected/differential diagnoses and infections).
- MEDICATION: specific drug names (e.g., meropenem, vancomycin). (Do NOT label tests or procedures.)
- DOSAGE: medication dose/regimen ONLY (e.g., "100 mg/kg/day", "in three doses", "IV", "once daily"). Must be tied to a MEDICATION in the same sentence/phrase.
- DURATION: time spans or timing of therapy/symptoms (e.g., "for 3 weeks", "after 3 weeks", "first week").
- SIDE_EFFECT: adverse effects/complications of treatment (e.g., leukopenia, neutropenia) when presented as developed during/after therapy.

STRICT EXCLUSIONS (do NOT extract):
- Imaging/procedures/tests: MRI, ultrasound, radiograph, CT, blood test

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[]
```
You returned correct json, but it contains not requested type
cannot parse json output: Extra data: line 3 column 1 (char 4)

You are NER model. Find all named entities in text (between tags <text> and </text).

Allowed types ONLY:
- SYMPTOM: patient-reported symptoms or clinical manifestations (e.g., pain, weakness, fever, inability to walk).
- DIAGNOSIS: diseases/conditions (including suspected/differential diagnoses and infections).
- MEDICATION: specific drug names (e.g., meropenem, vancomycin). (Do NOT label tests or procedures.)
- DOSAGE: medication dose/regimen ONLY (e.g., "100 mg/kg/day", "in three doses", "IV", "once daily"). Must be tied to a MEDICATION in the same sentence/phrase.
- DURATION: time spans or timing of therapy/symptoms (e.g., "for 3 weeks", "after 3 weeks", "first week").
- SIDE_EFFECT: adverse effects/complications of treatment (e.g., leukopenia, neutropenia) when presented as developed during/after therapy.

STRICT EXCLUSIONS (do NOT extract):
- Imagi

481    []
3      []
Name: ner_prompt, dtype: object

In [106]:
print('xa')

xa
